In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules



In [7]:
df = pd.read_csv("ratings_small.csv")
df


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [8]:
movies = pd.read_csv("movies_metadata.csv")
movies = movies[['id','title']]
movies = movies.dropna()
movies.columns = ['movieId','title']
movies = movies.astype({'movieId': 'int64'})
movies

C:\Users\marco\AppData\Local\Temp\ipykernel_26424\1379977588.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("movies_metadata.csv")


,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II
...,...,...
45461,439050,Subdue
45462,111109,Century of Birthing
45463,67758,Betrayal
45464,227506,Satan Triumphant


In [9]:
df = pd.merge(df, movies).drop(['movieId','timestamp'],axis='columns')
df

,userId,rating,title
0,1,2.5,Rocky III
1,1,1.0,Greed
2,1,4.0,American Pie
3,1,2.0,My Tutor
4,1,2.0,Jay and Silent Bob Strike Back
...,...,...,...
44989,671,4.0,Boogie Nights
44990,671,4.0,Waiter
44991,671,3.5,A Bridge Too Far
44992,671,4.5,The Last Laugh


In [10]:
movie_user_matrix = df.pivot_table(index='userId', columns='title',values='rating', fill_value=0)
movie_user_matrix[movie_user_matrix > 0] = 1
#movie_user_matrix= movie_user_matrix.reset_index()
movie_user_matrix.columns.name = ''
movie_user_matrix = movie_user_matrix.astype('int')
movie_user_matrix

,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
userId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
668,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df = movie_user_matrix.astype(bool)
df

,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
userId,,,,,,,,,,,,,,,,,,,,,
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
668,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
669,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
frequent_itemsets.sort_values(by='support',ascending=False)

,support,itemsets
111,0.482861,(Terminator 3: Rise of the Machines)
130,0.463487,(The Million Dollar Hotel)
105,0.454545,(Solaris)
113,0.433681,(The 39 Steps)
69,0.408346,(Monsoon Wedding)
...,...,...
7315,0.101341,"(Monsoon Wedding, Titanic, Terminator 3: Rise ..."
7313,0.101341,"(Monsoon Wedding, Terminator 3: Rise of the Ma..."
7312,0.101341,"(Monsoon Wedding, Titanic, Rain Man, Terminato..."
7288,0.101341,"(Three Colors: Red, The Hours, Terminator 3: R..."


In [17]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2)
sorted_rules = rules.sort_values('lift', ascending=False)
sorted_rules[["antecedents", "consequents", "lift"]]

,antecedents,consequents,lift
637,(Waiter),(Muxmäuschenstill),5.601529
636,(Muxmäuschenstill),(Waiter),5.601529
28340,"(Big Fish, Titanic)","(Psycho, Rain Man)",5.133663
28345,"(Psycho, Rain Man)","(Big Fish, Titanic)",5.133663
28371,"(Solaris, Psycho)","(Big Fish, Titanic)",5.093399
...,...,...,...
39514,(Solaris),"(Sissi, Titanic, The Passion of Joan of Arc)",2.000000
31064,(Solaris),"(Men in Black II, Terminator 3: Rise of the Ma...",2.000000
39507,"(Sissi, Titanic, The Passion of Joan of Arc)",(Solaris),2.000000
57589,"(The Hours, Terminator 3: Rise of the Machines...",(Solaris),2.000000


In [19]:
sorted_rules.dtypes

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
zhangs_metric         float64
dtype: object

In [20]:
df_MIB = sorted_rules[sorted_rules['antecedents'].apply(lambda x: len(x) ==1 and next(iter(x)) == 'Men in Black II')]
df_MIB = df_MIB[df_MIB['lift'] > 2]
df_MIB.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
8610,(Men in Black II),"(Scarface, Terminator 3: Rise of the Machines)",0.33383,0.137109,0.107303,0.321429,2.344332,0.061531,1.271629,0.860800
31007,(Men in Black II),"(The Talented Mr. Ripley, Terminator 3: Rise o...",0.33383,0.129657,0.101341,0.303571,2.341338,0.058058,1.249723,0.859982
8654,(Men in Black II),"(The 39 Steps, The Talented Mr. Ripley)",0.33383,0.132638,0.101341,0.303571,2.288724,0.057063,1.245443,0.845243
29282,(Men in Black II),"(Dawn of the Dead, Solaris, Terminator 3: Rise...",0.33383,0.132638,0.101341,0.303571,2.288724,0.057063,1.245443,0.845243
30990,(Men in Black II),"(The 39 Steps, Terminator 3: Rise of the Machi...",0.33383,0.147541,0.111773,0.334821,2.269345,0.062520,1.281549,0.839642
